# Récupération et retraitement des données

## A) Récupération des données

Le projet repose sur deux sources de données principales:
1. Les données relatives aux diagnostics de performance énergétique (DPE) des logements. Source: Ademe.
2. Les données socio-démographiques caractéristiques des communes (dispositif Filosofi). Source: Insee.
Le premier jeu de données est importé via l'API mise à disposition par l'Ademe sur son portail Opendata. Le second jeu de données est lui importé via le package Pyinsee (permettant de disposer d'une version "propre" d'un grand nombre de fichiers de données de l'Insee).

### 1) Récupération du premier jeu de données via l'API publique "APE logements" (portail Open data de l'ADEME)

In [1]:
# Importations:
import requests
import pandas as pd
import geopandas as gpd

#### a) Récupération préalable de l'ensemble des codes communes situés dans la région Ile-de-france:

In [2]:
# Recherche de la Liste des codes de communes

co_com = pd.read_csv("https://www.insee.fr/fr/statistiques/fichier/6051727/commune_2022.csv")
print(co_com.head())

# Sélection des communes de la région ile-de-France
CODES = co_com.loc[co_com["REG"]==11,"COM"].unique()
co_com2 = co_com.loc[co_com["REG"]==11,]
print(len(co_com2))
print(len(CODES))  #1288

  TYPECOM    COM   REG DEP CTCD  ARR  TNCC                    NCC  \
0     COM  01001  84.0  01  01D  012     5  ABERGEMENT CLEMENCIAT   
1     COM  01002  84.0  01  01D  011     5    ABERGEMENT DE VAREY   
2     COM  01004  84.0  01  01D  011     1      AMBERIEU EN BUGEY   
3     COM  01005  84.0  01  01D  012     1    AMBERIEUX EN DOMBES   
4     COM  01006  84.0  01  01D  011     1                AMBLEON   

                  NCCENR                  LIBELLE   CAN  COMPARENT  
0  Abergement-Clémenciat  L'Abergement-Clémenciat  0108        NaN  
1    Abergement-de-Varey    L'Abergement-de-Varey  0101        NaN  
2      Ambérieu-en-Bugey        Ambérieu-en-Bugey  0101        NaN  
3    Ambérieux-en-Dombes      Ambérieux-en-Dombes  0122        NaN  
4                Ambléon                  Ambléon  0104        NaN  
1288
1288


#### b) Définition des fonctions permettant la connexion à l'API et la récupération des données selon les termes clés

In [3]:
# Définition de la fonction d'extraction de données

def get_dpe_from_url(url):

    req = requests.get(url)
    wb = req.json()
    try:
        df = pd.json_normalize(wb["results"]) # Conversion du fichier Json en dataframe pandas
    except json.JSONDecodeError:
        print("A file was empty")
    
    if 'longitude' in df.columns.tolist() and 'latitude' in df.columns.tolist():
    # Conversion du dataframe classique en dataframe géographique 
        dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["longitude"], df["latitude"]), crs = 4326) 
        dpe = dpe.dropna(subset = ['longitude', 'latitude'])

        return dpe

In [4]:
# Fixation des paramètres pour le requêtage
    #code_commune="01450"
code_commune = CODES[53]
size = 10000
root="https://data.ademe.fr/data-fair/api/v1/datasets/dpe-france/lines"
url_api = f"{root}?page=1&after=100&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + f"&size={size}&select=" + "%2A&sampling=max"
url_api2 = f"{root}?after=100&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + f"&size={size}&select=" + "%2A&sampling=neighbors"


In [5]:
# Fonction qui retourne l'url à partir du code_commune
def ret_url (cod,size_):
    return f"{root}?after=100&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{cod}" + "%22" + f"&size={size_}&select=" + "%2A&sampling=neighbors"


### c) Récupération des données relatives à l'ensemble des DPE réalisés en Ile-de-France et mises à disposition dans le portail Open data de l'Ademe

En requêtant l'API "DPE logements", on récupère tous les DPE disponibles pour chaque commune d'Ile-de-France puis on les unifie en un seul dataframe "dpe_f".

In [6]:
# Construction d'un dataframe complet des données pour la Région île-de-France

dpe_f = get_dpe_from_url(ret_url(CODES[0],10000))

for i in range(len(CODES)-1):                                                  # len(CODES)
    
    dpei = get_dpe_from_url(ret_url(CODES[i+1],10000))
    if dpei is not None:
        Commune = co_com2.loc[co_com2["COM"]==CODES[i],"NCC"].tolist()[0]
        dpei["Commune"] = Commune
        #dpei.set_index("Commune")
        dpe_f = pd.concat([dpe_f, dpei])
    


In [7]:
dpe_f.head()

,classe_consommation_energie,tr001_modele_dpe_type_libelle,annee_construction,_geopoint,latitude,surface_thermique_lot,_i,tr002_type_batiment_description,geo_adresse,_rand,...,version_methode_dpe,nom_methode_dpe,tv016_departement_code,consommation_energie,date_etablissement_dpe,longitude,_score,_id,geometry,Commune
0,F,Vente,1970,"48.829392,2.374836",48.829392,80.00,101,Logement,15 Rue de Tolbiac 75013 Paris,531934,...,V2012,3CL - DPE,75,434.00,2013-05-09,2.374836,None,T7b7O-zAyMijXFdlyWl7M,POINT (2.37484 48.82939),NaN
1,F,Vente,1970,"48.829392,2.374836",48.829392,80.00,108,Logement,15 Rue de Tolbiac 75013 Paris,457143,...,V2012,3CL - DPE,75,434.00,2013-05-11,2.374836,None,99qI3IKDlsdqj_LgNBzmX,POINT (2.37484 48.82939),NaN
2,F,Vente,1970,"48.829392,2.374836",48.829392,80.00,109,Logement,15 Rue de Tolbiac 75013 Paris,885078,...,V2012,3CL - DPE,75,434.00,2013-05-11,2.374836,None,RIrNiQLnIG9Iqsg19e8aC,POINT (2.37484 48.82939),NaN
3,D,Vente,1940,"48.847662,2.28565",48.847662,29.00,119,Logement,11 Rue Beaugrenelle 75015 Paris,700764,...,V2012,3CL - DPE,75,170.02,2013-05-13,2.285650,None,XZIHgdlUOrEuAIrcpa9xa,POINT (2.28565 48.84766),NaN
4,N,Vente,1900,"48.839935,2.332912",48.839935,6.99,123,Logement,11 Rue Campagne Première 75014 Paris,215124,...,V2012,3CL - DPE,75,0.00,2013-05-13,2.332912,None,uVKCNovGIFXlkwZwxcXWD,POINT (2.33291 48.83993),NaN


In [8]:
dpe_f.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1323802 entries, 0 to 23
Data columns (total 24 columns):
 #   Column                           Non-Null Count    Dtype   
---  ------                           --------------    -----   
 0   classe_consommation_energie      1323802 non-null  object  
 1   tr001_modele_dpe_type_libelle    1323802 non-null  object  
 2   annee_construction               1323802 non-null  int64   
 3   _geopoint                        1323802 non-null  object  
 4   latitude                         1323802 non-null  float64 
 5   surface_thermique_lot            1323726 non-null  float64 
 6   _i                               1323802 non-null  int64   
 7   tr002_type_batiment_description  1323802 non-null  object  
 8   geo_adresse                      1323802 non-null  object  
 9   _rand                            1323802 non-null  int64   
 10  code_insee_commune_actualise     1323802 non-null  object  
 11  estimation_ges                   1

### 2) Récupération du second jeu de données via le package Pynsee: données socio-démographiques sur les communes (source: dispositif Filosofi, Insee)

#### a) Installation du package pynsee disponible seulement sous Github

In [9]:
!pip install pathlib2
!pip install python-Levenshtein
!pip install --upgrade xlrd
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.1 MB/s eta 0:00:00
  Cloning https://github.com/InseeFrLab/Py-Insee-Data.git to /tmp/pip-req-build-pi1sbsy_
  Running command git clone --filter=blob:none --quiet https://github.com/InseeFrLab/Py-Insee-Data.git /tmp/pip-req-build-pi1sbsy_
  Resolved https://github.com/InseeFrLab/Py-Insee-Data.git to commit e74df7d0125aeba38402b735af7872b2c381d803
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 13.5 MB/s eta 0:00:00
  Created wheel for pynsee: filename=pynsee-0.1.2-py3-none-any.whl size=9805116 sha256=ee15768942bda7a9abda15cfe27b468c4a7ea30a5b26a2e41be0f6494f900e3e
  Stored in directory: /tmp/pip-ephem-wheel-cache-uy8

In [10]:
# Importation du package:
import pynsee
import pynsee.download

#### b) Récupération du fichier Filosofi 2016 au niveau communal

In [11]:
Filosofi = pynsee.download.download_file("FILOSOFI_COM_2016")

Downloading: 100%|██████████| 2.67M/2.67M [00:00<00:00, 8.09MiB/s]
Extracting: 100%|██████████| 10.6M/10.6M [00:00<00:00, 123MB/s]


In [12]:
Filosofi.head()

,CODGEO,LIBGEO,NBMENFISC16,NBPERSMENFISC16,MED16,PIMP16,TP6016,TP60AGE116,TP60AGE216,TP60AGE316,...,PPEN16,PPAT16,PPSOC16,PPFAM16,PPMINI16,PPLOGT16,PIMPOT16,D116,D916,RD16
0,01001,L'Abergement-Clémenciat,313,795.5,22679,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01002,L'Abergement-de-Varey,101,248,24382.083333333336,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01004,Ambérieu-en-Bugey,6363,14228,19721,49,17,19,22,22,...,27,8.8,6.9,2.8,2.1,2,-15.7,10457.083333333334,33880.555555555555,3.239962279688143
3,01005,Ambérieux-en-Dombes,633,1662.5,23378,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01006,Ambléon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
Filosofi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34932 entries, 0 to 34931
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CODGEO           34932 non-null  object
 1   LIBGEO           34932 non-null  object
 2   NBMENFISC16      31405 non-null  object
 3   NBPERSMENFISC16  31405 non-null  object
 4   MED16            31405 non-null  object
 5   PIMP16           5293 non-null   object
 6   TP6016           4372 non-null   object
 7   TP60AGE116       759 non-null    object
 8   TP60AGE216       1414 non-null   object
 9   TP60AGE316       1826 non-null   object
 10  TP60AGE416       1292 non-null   object
 11  TP60AGE516       890 non-null    object
 12  TP60AGE616       403 non-null    object
 13  TP60TOL116       2155 non-null   object
 14  TP60TOL216       3174 non-null   object
 15  PACT16           5293 non-null   object
 16  PTSA16           5293 non-null   object
 17  PCHO16           5293 non-null 

## B) Retraitement des données

### 1) Sélection des variables pertinentes pour l'étude

#### a) Dans le dataframe dpe_f

In [14]:
dpe_f=dpe_f.loc[:,["_id","date_etablissement_dpe","consommation_energie","classe_consommation_energie","estimation_ges","classe_estimation_ges","annee_construction","surface_thermique_lot","Commune","code_insee_commune_actualise","longitude","latitude","tr002_type_batiment_description","tv016_departement_code","_geopoint","geometry"]]
dpe_f.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1323802 entries, 0 to 23
Data columns (total 16 columns):
 #   Column                           Non-Null Count    Dtype   
---  ------                           --------------    -----   
 0   _id                              1323802 non-null  object  
 1   date_etablissement_dpe           1323802 non-null  object  
 2   consommation_energie             1323802 non-null  float64 
 3   classe_consommation_energie      1323802 non-null  object  
 4   estimation_ges                   1323802 non-null  float64 
 5   classe_estimation_ges            1323802 non-null  object  
 6   annee_construction               1323802 non-null  int64   
 7   surface_thermique_lot            1323726 non-null  float64 
 8   Commune                          1313850 non-null  object  
 9   code_insee_commune_actualise     1323802 non-null  object  
 10  longitude                        1323802 non-null  float64 
 11  latitude                         1

On se concentre ici sur les seules variables en lien direct avec la performance énergétique des logements, ainsi que les variables permettant de géolocaliser précisément chaque logement.

#### b) Dans le dataframe Filosofi

In [15]:
Filosofi=Filosofi.loc[:,["CODGEO","LIBGEO","NBMENFISC16","NBPERSMENFISC16","MED16"]]
Filosofi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34932 entries, 0 to 34931
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CODGEO           34932 non-null  object
 1   LIBGEO           34932 non-null  object
 2   NBMENFISC16      31405 non-null  object
 3   NBPERSMENFISC16  31405 non-null  object
 4   MED16            31405 non-null  object
dtypes: object(5)
memory usage: 1.3+ MB


Cette sélection de variables est assez restrictive, mais il s'avère que les autres variables socio-économiques ne sont renseignées que pour les plus grosses communes.

### 2) Nettoyage et mise en forme de certaines variables

#### a) Cleaning de la variable "code_commune_actualise" dans le df "dpe_f"

En examinant les données, il est apparu que la variable "code_commune_actualise" comportait un certain nombre de modalités mal renseignées: des codes commune de moins de 5 chiffres,des codes communes comportant le signe ".", des codes commune comportant des noms de commune, etc. Aussi un nettoyage de cette variable clé s'impose: 

In [16]:

# liste des codes commune exisistant initialement dans le fichier DPE brut:
a=dpe_f["code_insee_commune_actualise"].unique().tolist()

# on transforme les éléments de la liste en str:
b=[str(x) for x in a]

# on code le filtre de "nettoyage" à appliquer à notre df "dpe_f":
filtre = [x for x in b if len(x) == 5 if "." not in x if x[:2] in (["75","77","91","92","93","94","95"])]
# on ne garde que les code commune composés de 5 chiffres, sans "." et débutant par les nombres suivants:
# 75, 77, 91, 92, 93, 94 et 95.

print(len(filtre)) # nombre de codes commune retenus in fine pour l'IDF.

# Application du filtre au df "dpe_f":
dpe_f2=dpe_f[dpe_f["code_insee_commune_actualise"].isin(filtre)].copy()
# tous les codes commune sont clean dans "dpe_f2".
dpe_f2.shape

1027


(1134380, 16)

In [17]:
dpe_f.shape # on a donc supprimé 189 422 lignes du df "dpe_f" suite au nettoyage des codes commune.

(1323802, 16)

#### b) Dans le df "Filosofi", passage au format numérique des variables "NBMENFISC16", "NBPERSMENFISC16" et "MED16"  

In [19]:
# Modification des types des variables: passage au format numérique de toutes les variables sauf codgeo et libgeo.
liste_var=Filosofi.columns.tolist()[2:]
for var in liste_var:
    Filosofi[var]=pd.to_numeric(Filosofi[var])
Filosofi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34932 entries, 0 to 34931
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CODGEO           34932 non-null  object 
 1   LIBGEO           34932 non-null  object 
 2   NBMENFISC16      31405 non-null  float64
 3   NBPERSMENFISC16  31405 non-null  float64
 4   MED16            31405 non-null  float64
dtypes: float64(3), object(2)
memory usage: 1.3+ MB


#### c) Subsetting du df "Filosofi" en ne gardant que les codes commune d'Ile-de-France

In [21]:
# Création d'une variable "département":
Filosofi["DEP"]=Filosofi["CODGEO"].str[:2]
# Filtrage géographique: on ne garde que les départements d'Ile-de-France:
Filosofi_IDF=Filosofi[Filosofi["DEP"].isin(["75","91","92","93","94","95","77","78"])].reset_index(drop=True)

In [22]:
Filosofi_IDF.head()

,CODGEO,LIBGEO,NBMENFISC16,NBPERSMENFISC16,MED16,DEP
0,75056,Paris,1027657.0,2074629.5,26808.000000,75
1,75101,Paris 1er Arrondissement,8483.0,15477.0,32697.333333,75
2,75102,Paris 2e Arrondissement,11362.0,20563.0,30566.500000,75
3,75103,Paris 3e Arrondissement,17727.0,32292.5,31333.000000,75
4,75104,Paris 4e Arrondissement,13955.0,25585.5,31007.222222,75


### 3) Création de nouvelles variables utiles pour l'analyse statistique des données

#### a) Dans le df dpe_f2:

In [23]:
# Calcul de nouvelles variables dans dpe_f: année du DPE, âge du logement, département.

# 1) Création d'une variable "année" de réalisation du DPE:
dpe_f2["annee"]=dpe_f2.date_etablissement_dpe.str[:4]

# 2) Création d'une variable "âge" du logement:
# il faut d'abord transformer la variable "année" en int64:
dpe_f2["annee2"]=pd.to_numeric(dpe_f2["annee"])
dpe_f2["age_logement"]=dpe_f2["annee2"]-dpe_f2["annee_construction"]

# 3) Création d'une variable "département" à partir du code de la commune du DPE:
dpe_f2["dep"]=dpe_f2.code_insee_commune_actualise.str[:2]

# Discrétisation de variables quantitatives:

# Classes d'âge:
dpe_f2['classe_age'] = pd.cut(dpe_f2['age_logement'],bins=[dpe_f2['age_logement'].min(),5,10,25,50,100,
                                                          dpe_f2['age_logement'].max()],\
                                      labels=["Neuf (<5ans)","Récent (entre 5 et 10 ans)",\
                                              "Entre 10 et 25 ans","Entre 25 et 50 ans",\
                                              "Entre 50 et 100 ans","Plus de 100 ans"],\
                                     include_lowest=True)
# Classes de surface thermique: 
dpe_f2['classe_surface'] = pd.cut(dpe_f2['surface_thermique_lot'],bins=[dpe_f2['surface_thermique_lot'].min(),
                                                            30,60,100,dpe_f2['surface_thermique_lot'].max()],\
                                                          labels=["Moins de 30 m2",
                                                                  "Entre 30 et 60 m2",
                                              "Entre 60 et 100 m2","Plus de 100 m2"],\
                                     include_lowest=True)

# Quintiles de niveaux de conso d'énergie:
dpe_f2["conso_energie_quintiles"]=pd.qcut(dpe_f2["consommation_energie"],q=5,labels=["Q1","Q2","Q3","Q4","Q5"])

# Quintiles de niveaux d'émission de GES:
dpe_f2["estimation_ges_quintiles"]=pd.qcut(dpe_f2["estimation_ges"],q=5,labels=["Q1","Q2","Q3","Q4","Q5"])


In [24]:
dpe_f2.head()

,_id,date_etablissement_dpe,consommation_energie,classe_consommation_energie,estimation_ges,classe_estimation_ges,annee_construction,surface_thermique_lot,Commune,code_insee_commune_actualise,...,_geopoint,geometry,annee,annee2,age_logement,dep,classe_age,classe_surface,conso_energie_quintiles,estimation_ges_quintiles
0,T7b7O-zAyMijXFdlyWl7M,2013-05-09,434.00,F,25.00,D,1970,80.00,NaN,75056,...,"48.829392,2.374836",POINT (2.37484 48.82939),2013,2013,43,75,Entre 25 et 50 ans,Entre 60 et 100 m2,Q5,Q4
1,99qI3IKDlsdqj_LgNBzmX,2013-05-11,434.00,F,25.00,D,1970,80.00,NaN,75056,...,"48.829392,2.374836",POINT (2.37484 48.82939),2013,2013,43,75,Entre 25 et 50 ans,Entre 60 et 100 m2,Q5,Q4
2,RIrNiQLnIG9Iqsg19e8aC,2013-05-11,434.00,F,25.00,D,1970,80.00,NaN,75056,...,"48.829392,2.374836",POINT (2.37484 48.82939),2013,2013,43,75,Entre 25 et 50 ans,Entre 60 et 100 m2,Q5,Q4
3,XZIHgdlUOrEuAIrcpa9xa,2013-05-13,170.02,D,39.78,E,1940,29.00,NaN,75056,...,"48.847662,2.28565",POINT (2.28565 48.84766),2013,2013,73,75,Entre 50 et 100 ans,Moins de 30 m2,Q3,Q4
4,uVKCNovGIFXlkwZwxcXWD,2013-05-13,0.00,N,0.00,N,1900,6.99,NaN,75056,...,"48.839935,2.332912",POINT (2.33291 48.83993),2013,2013,113,75,Plus de 100 ans,Moins de 30 m2,Q1,Q1


#### b) Dans le df Filosofi_IDF

In [25]:
# Discrétisation de la variable "MED16": niveau de vie médian de la commune en 2016
Filosofi_IDF["MED16_quintiles"]=pd.qcut(Filosofi_IDF["MED16"],q=5,labels=["Q1","Q2","Q3","Q4","Q5"])

### 4) Agrégation des variables du df "dpe_f2" par commune

#### a) Comptage préalable des dpe par modalité des différentes variables catégorielles

- Variable "classe de consommation énergétique des logements"(classe_consommation_energie):

In [26]:
dpe_f2["classe_consommation_energie"].value_counts()

D    262922
B    217961
E    217040
N    156347
C    116650
F     80697
A     53212
G     29550
H         1
Name: classe_consommation_energie, dtype: int64

On peut observer que 156 348 dpe sont mal ou pas renseignés (catégories N et H). Par conséquent, nous procédons à un filtrage de dpe_f2 pour exclure ces observations de l'analyse:

In [27]:
dpe_f3=dpe_f2[dpe_f2["classe_consommation_energie"].isin(["A","B","C","D","E","F","G"])].reset_index(drop=True)

In [28]:
dpe_f3.shape

(978032, 24)

In [29]:
dpe_f2.shape

(1134380, 24)

On a ainsi exclu 156 348 dpe avec une classe de consommation énergétique mal ou pas renseignée.

- Variable "classe d'émission de gaz à effet de serre des logements"(classe_estimation_ges):

In [30]:
dpe_f3["classe_estimation_ges"].value_counts()

C    306023
E    170906
D    150697
B    143414
F     94861
A     71886
G     40244
N         1
Name: classe_estimation_ges, dtype: int64

Sur la base du df "dpe_f3", on constate qu'un seul dpe avec une classe d'émission de ges mal renseignée. On l'exclut de l'analyse:

In [31]:
dpe_f4=dpe_f3[dpe_f3["classe_estimation_ges"].isin(["A","B","C","D","E","F","G"])].reset_index(drop=True)

- Variable "type de logement" (tr002_type_batiment_description):

In [32]:
dpe_f4["tr002_type_batiment_description"].value_counts()

Logement                                             641451
Maison Individuelle                                  228335
Bâtiment collectif à usage principal d'habitation    108245
Name: tr002_type_batiment_description, dtype: int64

On constate que pour la grande majorité des DPE, le type de logement n'est pas renseigné, ce qui rend peu opératoire cette variable dans la suite de l'anlyse.

- Variable "Classes d'âge du logement" (classe_age):

In [33]:
dpe_f4["classe_age"].value_counts()

Entre 25 et 50 ans            283165
Entre 50 et 100 ans           256253
Neuf (<5ans)                  251017
Entre 10 et 25 ans            101727
Plus de 100 ans                54057
Récent (entre 5 et 10 ans)     31812
Name: classe_age, dtype: int64

- Variable "Classes de surface thermique du logement" (classe_surface):

In [34]:
dpe_f4["classe_surface"].value_counts()

Entre 60 et 100 m2    372427
Entre 30 et 60 m2     309215
Plus de 100 m2        177272
Moins de 30 m2        119073
Name: classe_surface, dtype: int64

#### b) Transformation des variables catégorielles en indicatrices

In [35]:
dpe_IDF=pd.get_dummies(data=dpe_f4, columns=['classe_consommation_energie','classe_estimation_ges',
                                    'classe_age','classe_surface','conso_energie_quintiles',
                                    'estimation_ges_quintiles'])
dpe_IDF.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 978031 entries, 0 to 978030
Data columns (total 52 columns):
 #   Column                                 Non-Null Count   Dtype   
---  ------                                 --------------   -----   
 0   _id                                    978031 non-null  object  
 1   date_etablissement_dpe                 978031 non-null  object  
 2   consommation_energie                   978031 non-null  float64 
 3   estimation_ges                         978031 non-null  float64 
 4   annee_construction                     978031 non-null  int64   
 5   surface_thermique_lot                  977987 non-null  float64 
 6   Commune                                970266 non-null  object  
 7   code_insee_commune_actualise           978031 non-null  object  
 8   longitude                              978031 non-null  float64 
 9   latitude                               978031 non-null  float64 
 10  tr002_type_batiment_description     

#### c) Agrégation des principales variables numériques par commune (moyenne/médiane):

In [36]:
liste_var=dpe_IDF.columns.tolist()

# 1) on sélectionne les variables dont on veut calculer la moyenne par commune (+ le code commune):
var_filtre1=[var for var in liste_var if var[:6]=="classe" or var[:5]=="conso" or var[:10]=="estimation" \
            or var=="surface_thermique_lot" or var=="age_logement" or var=="code_insee_commune_actualise"]
dpe_IDF_filtre1=dpe_IDF[var_filtre1]
# agrégation: calcul de la moyenne par commune:
dpe_IDF_filtre1_com=dpe_IDF_filtre1.groupby('code_insee_commune_actualise').mean().reset_index()

#2) on sélectionne les variables dont on veut calculer la médiane par commune (+le code commune)
var_filtre2=['code_insee_commune_actualise','surface_thermique_lot','age_logement','consommation_energie',
            'estimation_ges']
dpe_IDF_filtre2=dpe_IDF[var_filtre2]
# agrégation: calcul de la moyenne par commune:
dpe_IDF_filtre2_com=dpe_IDF_filtre2.groupby('code_insee_commune_actualise').median().reset_index()
del dpe_IDF_filtre2_com['code_insee_commune_actualise']
dpe_IDF_filtre2_com=dpe_IDF_filtre2_com.rename(columns = {'surface_thermique_lot': 'surface_thermique_lot_median',
                                                         'age_logement':'age_logement_median',
                                                         'consommation_energie':'consommation_energie_median',
                                                         'estimation_ges':'estimation_ges_median'})

#3) on crée enfin une variable "nombre de dpe par commune":
var_filtre3=['code_insee_commune_actualise','_id']
dpe_IDF_filtre3=dpe_IDF[var_filtre3]
dpe_IDF_filtre3_com=dpe_IDF_filtre3.groupby('code_insee_commune_actualise').count().reset_index()
del dpe_IDF_filtre3_com['code_insee_commune_actualise']
dpe_IDF_filtre3_com=dpe_IDF_filtre3_com.rename(columns = {'_id': 'nombre_dpe'})

dpe_IDF_com=pd.concat([dpe_IDF_filtre1_com,dpe_IDF_filtre2_com,dpe_IDF_filtre3_com],axis=1)

In [37]:
dpe_IDF_com.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 44 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   code_insee_commune_actualise           1026 non-null   object 
 1   consommation_energie                   1026 non-null   float64
 2   estimation_ges                         1026 non-null   float64
 3   surface_thermique_lot                  1026 non-null   float64
 4   age_logement                           1026 non-null   float64
 5   classe_consommation_energie_A          1026 non-null   float64
 6   classe_consommation_energie_B          1026 non-null   float64
 7   classe_consommation_energie_C          1026 non-null   float64
 8   classe_consommation_energie_D          1026 non-null   float64
 9   classe_consommation_energie_E          1026 non-null   float64
 10  classe_consommation_energie_F          1026 non-null   float64
 11  clas

### 5) Jointure entre les dataframes "Filisofi" et "df_dpe_com": création du df "df_dpe_filo_com"

On enrichit le dataframe dpe_IDF_com avec les données socio-économiques des communes contenues dans le dataframe Filosofi_com. On procède à une jointure des deux dataframes:

In [38]:
Filosofi_dpe_IDF_com=pd.merge(Filosofi_IDF,dpe_IDF_com,left_on="CODGEO",right_on="code_insee_commune_actualise",how="left")
Filosofi_dpe_IDF_com.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1288 entries, 0 to 1287
Data columns (total 51 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   CODGEO                                 1288 non-null   object  
 1   LIBGEO                                 1288 non-null   object  
 2   NBMENFISC16                            1270 non-null   float64 
 3   NBPERSMENFISC16                        1270 non-null   float64 
 4   MED16                                  1270 non-null   float64 
 5   DEP                                    1288 non-null   object  
 6   MED16_quintiles                        1270 non-null   category
 7   code_insee_commune_actualise           1026 non-null   object  
 8   consommation_energie                   1026 non-null   float64 
 9   estimation_ges                         1026 non-null   float64 
 10  surface_thermique_lot                  1026 non-null   float

### 6) Export des dataframe finaux au format CSV

On sauvegarde les dataframes finaux sous forme de fichiers .csv dans le repository "data_science_project" de Github:

#### a) Export du dataframe dpe_IDF: ensemble de caractéristiques pour chaque DPE d'Ile-de-France

In [39]:
dpe_IDF.to_csv("~/work/data_science_project/dpe_IDF.csv", index=False,header=True)

#### b) Export du dataframe Filosofi_dpe_IDF_com: caractéristiques des communes d'Ile-de-France en terme de performance énergétique des logement et de niveau de vie des ménages

In [40]:
Filosofi_dpe_IDF_com.to_csv("~/work/data_science_project/Filosofi_dpe_IDF_com.csv", index=False,header=True)